In [9]:
import os
import numpy as np
import pandas as pd
import pickle as pkl
import imageio
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from PIL import Image
import PIL
from astropy.nddata.utils import Cutout2D
from scipy import ndimage
import time
import matplotlib.pyplot as plt

In [2]:
# define functions to normalize image
def normalize(arr):
    arr = np.array(arr)
    arr = arr - np.min(arr)
    arr = arr / np.max(arr)
    return np.array(arr)

def gaussian_normalize(arr):
    arr = np.array(arr)
    arr = arr - np.mean(arr)
    arr = arr / np.std(arr)
    return np.array(arr)

# Bubble or no bubble

In [3]:
df = pkl.load(open("FeaturesDataFrame.p", "rb"))
df = df.query('not (bubblecount == -1 & blobpeakfeature > 1)').copy()
df.reset_index(drop=True, inplace=True)
y = 1-(np.array(df['bubblecount'])<=0)
x = np.array(df[['edgefeature', 'blobfeature', 'blobpeakfeature']])
for i in range(3):
    x[:,i] = gaussian_normalize(x[:,i])
print(x.shape, y.shape)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(23334, 3) (23334,)
(18667, 3) (4667, 3) (18667,) (4667,)


In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation=tf.nn.relu, input_shape=(3,)),
    tf.keras.layers.Dense(2)
])

In [5]:
model.compile(optimizer = 'adam', 
               loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
               metrics = ["accuracy"])
model.fit(x_train,y_train,epochs = 20)
test_accuracy = model.evaluate(x_test, y_test, verbose = 3)
print("\nLoss: %.4f, Accuracy: %.4f"%tuple(test_accuracy))
y_pred = model.predict_classes(x_test)
print(confusion_matrix(y_test,y_pred))

Epoch 1/20
18667/18667 [==============================] - 0s 15us/sample - loss: 0.3318 - acc: 0.8829
Epoch 2/20
18667/18667 [==============================] - 0s 12us/sample - loss: 0.1687 - acc: 0.9501
Epoch 3/20
18667/18667 [==============================] - 0s 12us/sample - loss: 0.1602 - acc: 0.9540
Epoch 4/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.1559 - acc: 0.9558
Epoch 5/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.1527 - acc: 0.9570
Epoch 6/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.1501 - acc: 0.9582
Epoch 7/20
18667/18667 [==============================] - 0s 12us/sample - loss: 0.1479 - acc: 0.9594
Epoch 8/20
18667/18667 [==============================] - 0s 12us/sample - loss: 0.1462 - acc: 0.9598
Epoch 9/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.1448 - acc: 0.9601
Epoch 10/20
18667/18667 [==============================] - 0s 12us/sample - loss: 

# Number of bubbles

In [6]:
df = pkl.load(open("FeaturesDataFrame.p", "rb"))
df = df.query('not (bubblecount == -1 & blobpeakfeature > 1)').copy()
df.reset_index(drop=True, inplace=True)
y = np.array(df['bubblecount'])+1
x = np.array(df[['edgefeature', 'blobfeature', 'blobpeakfeature']])
for i in range(3):
    x[:,i] = gaussian_normalize(x[:,i])
print(x.shape, y.shape)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(23334, 3) (23334,)
(18667, 3) (4667, 3) (18667,) (4667,)


In [7]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(8, activation=tf.nn.relu, input_shape=(3,)),
    tf.keras.layers.Dense(6)
])

In [8]:
model.compile(optimizer = 'adam', 
               loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True),
               metrics = ["accuracy"])
model.fit(x_train,y_train,epochs = 20)
test_accuracy = model.evaluate(x_test, y_test, verbose = 3)
print("\nLoss: %.4f, Accuracy: %.4f"%tuple(test_accuracy))
y_pred = model.predict_classes(x_test)
print(confusion_matrix(y_test,y_pred))

Epoch 1/20
18667/18667 [==============================] - 0s 15us/sample - loss: 0.6829 - acc: 0.8748
Epoch 2/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.3046 - acc: 0.9365
Epoch 3/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.2677 - acc: 0.9401
Epoch 4/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.2454 - acc: 0.9411
Epoch 5/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.2271 - acc: 0.9414
Epoch 6/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.2140 - acc: 0.9433
Epoch 7/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.2083 - acc: 0.9469
Epoch 8/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.2056 - acc: 0.9478
Epoch 9/20
18667/18667 [==============================] - 0s 13us/sample - loss: 0.2040 - acc: 0.9483
Epoch 10/20
18667/18667 [==============================] - 0s 14us/sample - loss: 